# Tarea E-3. 

**Autores**: Jesús Martínez Leal y Carlos Sánchez Polo

**Enunciado**:

El algoritmo DOMINANT de `detección de anomalías` en grafos está basado en las ideas de redes convolucionales (GNN) y autoencoders (GAE) de los grafos.

- Introduce teóricamente las redes convolucionales en grafos (GNN).
- Aplica las GNN en ejemplos sencillos en los que se muestren los conceptos anteriormente introducidos.

## Conceptos previos

### Introducción a los grafos

Los grafos son estructuras matemáticas que nos permiten representar y modelar relaciones entre diversas entidades. En su forma más básica, un grafo está compuesto por un conjunto de `nodos` (también llamados vértices) unidos por enlaces llamados `aristas` (también llamados bordes). Típicamente, un grafo se representa como diagrama como un conjunto de círculos para los vértices, unido por líneas o curvas para las aristas. 

<figure align="center">
  <img src="./data/graph-theory-in-discrete-mathematics.png" alt="Figura de un grafo">
  <figcaption>Figura 1: Representación de un grafo.</figcaption>
</figure>

Los enlaces pueden ser `no dirigidos` o `dirigidos`, lo que significa que pueden representar relaciones unidireccionales o bidireccionales entre los nodos, respectivamente. Ejemplos de esto serían:

- **Enlaces no dirigidos**. Imaginemos que tenemos un grafo con vértices representando a personas en una fiesta y existe una conexión si dos personas se dan la mano. Las conexiones serán efectivamente no dirigidas dada la reprocidad que implica darse la mano.

- **Enlaces dirigidos**. Imaginemos ahora que el grafo cuenta con vértices representando las conexiones entre distintas estaciones de tren. Podría darse el caso que la estación A estuviera conectada con la estación B directamente, pero que no fuera al contrario.

<div style="text-align:center;">
  <figure style="display:inline-block;">
    <img src="./data/conexion_no_dirigida.png" alt="Conexión no dirigida">
    <figcaption>Figura 2: Enlace no dirigido.</figcaption>
  </figure>
  <figure style="display:inline-block;">
    <img src="./data/conexion_dirigida.png" alt="Conexión dirigida">
    <figcaption>Figura 3: Enlace dirigido.</figcaption>
  </figure>
</div>

Formalmente, un grafo $G$ es un `par ordenado` $G = (V, E)$, donde $V$ es un conjunto de nodos y $E$ un conjunto de aristas que relacionan estos. En un `grafo no dirigido` encontramos que todos los enlaces son no dirigidos, sucediendo justamente lo opuesto en un `grafo dirigido`. Por otra parte,  un `grafo mixto` permite los dos tipos de enlaces en su estructura, siendo un caso más general de los anteriores.



Cabe mencionar que los elementos individuales de cada grafo pueden almacenar información adicional:

- Información en nodos. La información almacenada en cada nodo puede incluir atributos para caracterizar mejor la entidad que representa.
- Información en enlaces. La información aquí puede incluir atributos que desciben la naturaleza de la relación entre los nodos.
- Información en el grafo en su conjunto. Información como el número total de nodos y aristas, densidad del grafo, etc.

<figure align="center">
  <img src="./data/information_graphs.png" alt="Información incrustada en un grafo">
  <figcaption>Figura 4: Información incrustada dentro de los elementos de un grafo.</figcaption>
</figure>


Los grafos constituyen herramientas poderosas y versátiles que se utilizan en amplia variedad de campos, debido a su capacidad de modelar y representar una amplia gama de relaciones y estructuras complejas.

### Introducción a las redes neuronales convolucionales